In [1]:
# common packages
import pandas as pd
import time

# # DL framework
import torch
from torchtext import data

from attractivedata import AttractiveData
from trainer import AttractiveTrainer

## Load and prepare data

In [2]:
train_file = 'data/train.csv'
test_file = 'data/test.csv'
pretrained_file = 'glove.42B.300d'
config = {
    'max_size': 64,
    'min_freq': 5,
    'batch_size': 64
}


In [3]:
AttractiveData = AttractiveData(train_file, test_file, pretrained_file, config)

In [4]:
for i, sentence in enumerate(AttractiveData.test_data):
    if i == 3:
        print(vars(AttractiveData.train_data[i]), vars(sentence))

{'Headline': ['Sorry', ',', 'i', 'spent', 'it', 'on', 'myself', '!', 'Harvey', 'Nichols', "'", 'hilarious', 'Christmas', 'advert', 'sees', 'people', 'treating', 'themselves', 'instead', 'of', 'others'], 'Category': 'femail', 'Label': '3.333333333333333'} {'Headline': ['Three', 'police', 'officers', 'accused', 'of', 'stealing', '?', '?', '30k', 'during', 'raid', 'on', 'criminal'], 'Category': 'news'}


In [5]:
len(AttractiveData.CATEGORIES_LABEL.vocab.freqs)

18

In [6]:
max_len = 0
for i in range(i):
    if len(AttractiveData.test_data[i].Headline) >= max_len:
        max_len = len(AttractiveData.test_data[i].Headline)
max_len

31

## Start to train

In [7]:
num_workers = 10

config['timestr'] = time.strftime("%Y%m%d-%H%M%S")
config['save_name'] = 'LSTM'
config['input_dim'] = len(AttractiveData.TEXT.vocab)
config['embedding_dim'] = 300
config['category_dim'] = len(AttractiveData.CATEGORIES_LABEL.vocab)
config['category_embedding_dim'] = 16
config['hidden_dim'] = 64
config['output_dim'] = 1
config['log_steps'] = 10
config['epochs'] = 100
config['lr'] = {
    'encoder': 1e-4,
    'embedding': 1e-5,
    'linear': 1e-4
}
config['num_layers'] = 2
config['nhead'] = 4
config['dropout'] = 0.1

pretrained_embeddings = AttractiveData.TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([1518, 300])


In [8]:
# max(AttractiveData.df_train.Headline.str.len()), max(AttractiveData.df_test.Headline.str.len())

In [9]:
AttractiveTrainer = AttractiveTrainer(config, AttractiveData.device, AttractiveData.trainloader, pretrained_embeddings)

In [10]:
AttractiveTrainer.model, AttractiveTrainer.total_params, AttractiveTrainer.total_learned_params

(AttractiveNet(
   (embedding): AttractiveEmbedding(
     (token): TokenEmbedding(1518, 300, padding_idx=1)
     (position): PositionalEmbedding()
     (dropout): Dropout(p=0.1, inplace=False)
   )
   (category_embedding): CategoryEmbedding(18, 16, padding_idx=0)
   (encoder): LSTM(300, 100, num_layers=2, dropout=0.1, bidirectional=True)
   (linear_lstm): Linear(in_features=200, out_features=16, bias=True)
   (linear_output): Linear(in_features=32, out_features=1, bias=True)
 ),
 1022137,
 566737)

In [11]:
AttractiveTrainer.train()

Epoch:   1%|          | 1/100 [00:00<00:58,  1.69it/s]
EP_train | avg_loss: 6.643013417720795 |
Epoch:   2%|▏         | 2/100 [00:01<00:55,  1.76it/s]
EP_train | avg_loss: 2.142767373472452 |
Epoch:   3%|▎         | 3/100 [00:01<00:54,  1.79it/s]
EP_train | avg_loss: 1.130288103595376 |
Epoch:   4%|▍         | 4/100 [00:02<00:52,  1.81it/s]
EP_train | avg_loss: 0.8910229504108429 |
Epoch:   5%|▌         | 5/100 [00:02<00:51,  1.85it/s]
EP_train | avg_loss: 0.8012351132929325 |
Epoch:   6%|▌         | 6/100 [00:03<00:50,  1.87it/s]
EP_train | avg_loss: 0.7534720655530691 |
Epoch:   7%|▋         | 7/100 [00:03<00:49,  1.87it/s]
EP_train | avg_loss: 0.7282735984772444 |
Epoch:   8%|▊         | 8/100 [00:04<00:48,  1.89it/s]
EP_train | avg_loss: 0.7081201877444983 |
Epoch:   9%|▉         | 9/100 [00:04<00:47,  1.90it/s]
EP_train | avg_loss: 0.6952569345012307 |
Epoch:  10%|█         | 10/100 [00:05<00:47,  1.89it/s]
EP_train | avg_loss: 0.6871952340006828 |
Epoch:  11%|█         | 11/100 [

## for classification, not better

In [14]:
# from sklearn.metrics import mean_squared_error
# a = AttractiveTrainer.train_predict
# AttractiveData.LABEL.vocab.itos[int(a[0])], AttractiveTrainer.train_true[0]
# correct = 0
# pred_list = []
# true_list = []
# for i in range(len(a)):
#     pred = AttractiveData.LABEL.vocab.itos[int(a[i])]
#     pred_list.append(float(pred))
#     true = AttractiveData.LABEL.vocab.itos[int(AttractiveTrainer.train_true[i])]
#     true_list.append(float(true))
# mean_squared_error(true_list, pred_list)
# # true_list

0.5601443355119825

## Below is testing

In [8]:
from transformermodel import TransformerModel
from attractivenet import AttractiveNet
PATH = './model/LSTM_20201030-214944_0.5417.100'
# load_model = TransformerModel(config).to(AttractiveData.device)
load_model = AttractiveNet(config).to(AttractiveData.device)
load_model.load_state_dict(torch.load(PATH))
load_model.eval()

AttractiveNet(
  (embedding): AttractiveEmbedding(
    (token): TokenEmbedding(1518, 300, padding_idx=1)
    (position): PositionalEmbedding()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (category_embedding): CategoryEmbedding(18, 16, padding_idx=0)
  (encoder): LSTM(300, 64, num_layers=2, dropout=0.1, bidirectional=True)
  (linear_output): Linear(in_features=144, out_features=1, bias=True)
)

In [9]:
def predict_attractive(sentence, category):
    indexed_sentence = [AttractiveData.TEXT.vocab.stoi[t] for t in sentence]
    indexed_category = [AttractiveData.CATEGORIES_LABEL.vocab.stoi[category]]
    tensor_sentence = torch.LongTensor(indexed_sentence).to(AttractiveData.device)
    tensor_category = torch.LongTensor(indexed_category).to(AttractiveData.device)

    tensor_sentence = tensor_sentence.unsqueeze(1)
    tensor_category = tensor_category

    prediction = load_model(tensor_sentence, tensor_category)
    
    return prediction

In [10]:
# train mean = 3.2, test mean = 2.8
predict_list = []
for i, sentence in enumerate(AttractiveData.test_data):
    prediction = predict_attractive(sentence.Headline, sentence.Category)
    # predict_list.append(prediction.item() - 3.2 + 2.8)
    predict_list.append(prediction.item())
AttractiveData.df_test['Label'] = predict_list
AttractiveData.df_test[['ID', 'Label']].to_csv(config['save_name'] + '.csv', index=False)

In [11]:
# train_category = list(AttractiveData.CATEGORIES_LABEL.vocab.freqs)
# test_category = list(AttractiveData.df_test['Category'].value_counts().keys())
# for each_test in test_category:
#     if each_test not in train_category:
#         print(each_test)
# print()
# for each_train in train_category:
#     if each_train not in test_category:
#         print(each_train)

## Below just for fun guess

In [12]:
import statistics
from sklearn.metrics import mean_squared_error

In [13]:
a = AttractiveData.df_train['Label'].to_list()
statistics.mean(a), statistics.stdev(a)

(3.150408496732026, 0.729501519321601)

In [14]:
statistics.mean(predict_list), statistics.stdev(predict_list)

(2.8167915543795683, 0.14611407210842048)

In [15]:
all_28 = [2.8] * len(predict_list)
mean_squared_error(all_28, predict_list)

0.021537228489634703